In [1]:
import gluoncv as gcv
import mxnet as mx
# mxnet NDarray 
from mxnet import nd

# MXNet Gluon Neural network layers.
from mxnet.gluon import nn

# optional for displaying the image
import matplotlib.pyplot as plt
import numpy as np
import os
import math

In [2]:
def build_rot_matrix(angle):
    #  a  b
    #  c  d
    
    # calculate terms
    a = math.cos(x)
    b = -math.sin(x)
    c = math.sin(x)
    d = math.cos(x)

    # print("a = ", a)
    # print("b = ", b)
    # print("c = ", c)
    # print("d = ", d)
    
    # build np array
    np_array = np.array([
        [a,b],
        [c,d]
    ])
    
    # build mx.nd array
    nd_array = mx.nd.array(np_array)
    
    return nd_array

In [3]:
x=0.5*math.pi

rotate_90 = build_rot_matrix(x)
print('rotate by 90 nd array is ', rotate_90)

rotate by 90 nd array is  
[[ 6.123234e-17 -1.000000e+00]
 [ 1.000000e+00  6.123234e-17]]
<NDArray 2x2 @cpu(0)>


In [4]:
# Input Image 
in_img_batch = nd.arange(9).reshape((1,3,3))
print('in_img_batch is ', in_img_batch)
in_img = in_img_batch[0]
print('in_img ', in_img)

in_img_batch is  
[[[0. 1. 2.]
  [3. 4. 5.]
  [6. 7. 8.]]]
<NDArray 1x3x3 @cpu(0)>
in_img  
[[0. 1. 2.]
 [3. 4. 5.]
 [6. 7. 8.]]
<NDArray 3x3 @cpu(0)>


In [5]:
# Output Image
# setup an ouptut array with all nines, so I can see overwrites with new values
out_img = nd.ones(9).reshape((3,3))*9
# out_img also needs to be in batch form
out_img_batch = nd.expand_dims(out_img, axis=0)
print('out_img_batch', out_img_batch)
print('out_img is ', out_img)

out_img_batch 
[[[9. 9. 9.]
  [9. 9. 9.]
  [9. 9. 9.]]]
<NDArray 1x3x3 @cpu(0)>
out_img is  
[[9. 9. 9.]
 [9. 9. 9.]
 [9. 9. 9.]]
<NDArray 3x3 @cpu(0)>


In [8]:
# NDArrayIter(data, label=None, batch_size=1, shuffle=False, 
#            last_batch_handle='pad', data_name='data', 
#            label_name='softmax_label')
#
# Ignore the label parameter.
dataiter = mx.io.NDArrayIter(in_img_batch, batch_size=1, shuffle=False, last_batch_handle='discard')
#batch_index = [0]
for batch in dataiter:
    
    print('loop entry - a single batch - a single image in batch.data[0] from what is in in_data')
    # Does this copy or get an alias to the input image?
    a_img_batch = batch.data[0]
    print ('a_img_batch = ', a_img_batch)
    print ('a_img_batch.shape = ', a_img_batch.shape)
    
    a_img_indexes = mx.nd.contrib.index_array(a_img_batch, axes=(1, 2))
    print('a_img_indexes  ', a_img_indexes)
    print ('a_img_indexes.shape ', a_img_indexes.shape)
    
    
    
    # Try to assign input data to output data based upon indicies
    orig_indexes = mx.nd.reshape(a_img_indexes, shape=(9,2))
    print('orig_indexes ', orig_indexes)
    orig_indexes = orig_indexes.astype("float32")
    
    # I've seen one variant where matrix-matrix multiply 
    # is done with .T.  why?
    new_indexes = nd.dot(orig_indexes, rotate_90) 
    print('new_indexes = ', new_indexes)
    new_indexes = new_indexes.astype('int64')
    print('new_indexes = ', new_indexes)
    new_indexes = new_indexes + nd.array(nd.array([0, 2])).astype('int64')
    print('new_indexes after shift to positive', new_indexes)

    print('out_img ', out_img)
    print('out_img.shape ', out_img.shape)
    out_img = a_img_batch[0][new_indexes[:,0],new_indexes[:,1]]
    print('out_img ', out_img)
    out_img = out_img.reshape(3,3)
    print('out_img ', out_img)


loop entry - a single batch - a single image in batch.data[0] from what is in in_data
a_img_batch =  
[[[0. 1. 2.]
  [3. 4. 5.]
  [6. 7. 8.]]]
<NDArray 1x3x3 @cpu(0)>
a_img_batch.shape =  (1, 3, 3)
a_img_indexes   
[[[[0 0]
   [0 1]
   [0 2]]

  [[1 0]
   [1 1]
   [1 2]]

  [[2 0]
   [2 1]
   [2 2]]]]
<NDArray 1x3x3x2 @cpu(0)>
a_img_indexes.shape  (1, 3, 3, 2)
orig_indexes  
[[0 0]
 [0 1]
 [0 2]
 [1 0]
 [1 1]
 [1 2]
 [2 0]
 [2 1]
 [2 2]]
<NDArray 9x2 @cpu(0)>
new_indexes =  
[[ 0.0000000e+00  0.0000000e+00]
 [ 1.0000000e+00  6.1232343e-17]
 [ 2.0000000e+00  1.2246469e-16]
 [ 6.1232343e-17 -1.0000000e+00]
 [ 1.0000000e+00 -1.0000000e+00]
 [ 2.0000000e+00 -1.0000000e+00]
 [ 1.2246469e-16 -2.0000000e+00]
 [ 1.0000000e+00 -2.0000000e+00]
 [ 2.0000000e+00 -2.0000000e+00]]
<NDArray 9x2 @cpu(0)>
new_indexes =  
[[ 0  0]
 [ 1  0]
 [ 2  0]
 [ 0 -1]
 [ 1 -1]
 [ 2 -1]
 [ 0 -2]
 [ 1 -2]
 [ 2 -2]]
<NDArray 9x2 @cpu(0)>
new_indexes after shift to positive 
[[0 2]
 [1 2]
 [2 2]
 [0 1]
 [1 1]
 [2 1]
 